In [28]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 250
num_epochs = 200
learning_rate = 1e-5
samples = 30000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (28, 28), color = 'white')
        #img = np.reshape(img,(28,28,1))
        #print(np.shape(img))
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([0,0,5,5],fill='red',outline='red')
        #im2_pos1.show()
        #print(np.shape(np.transpose(np.asarray(im2_pos1),(2,0,1))))
        im2_pos1 = np.reshape(im2_pos1,(28,28,3))
        #print(np.shape(im2_pos1))
        data1.append(np.array(np.transpose(np.asarray(im2_pos1),(2,0,1)),dtype=np.float32))

        img_1 = Image.new('RGB', (28, 28), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw_1 = ImageDraw.Draw(im2_pos2)
        draw_1.rectangle([15,15,20,20],fill='red',outline='red')
        im2_pos2 = np.reshape(im2_pos2,(28,28,3))
        data2.append(np.array(np.transpose(np.asarray(im2_pos2),(2,0,1)),dtype=np.float32))
    elif switch==1:
        img_blank = Image.new('RGB', (28, 28), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        im2_blank_pos1 = np.reshape(im2_blank_pos1,(28,28,3))
        data1.append(np.array(np.transpose(np.asarray(im2_blank_pos1),(2,0,1)),dtype=np.float32))

        img_blank_1 = Image.new('RGB', (28, 28), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        im2_blank_pos2 = np.reshape(im2_blank_pos2,(28,28,3))
        data2.append(np.array(np.transpose(np.asarray(im2_blank_pos2),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (28,28), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
im2 = np.reshape(im2,(28,28,3))
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 1)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        #out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        #out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))
    print("b:", model(b.reshape(1,3,28,28)))
    


(30000, 3, 28, 28)
epoch [1/200], loss:769.7832
x:  tensor([[8.7937]], grad_fn=<AddmmBackward>)
y: tensor([[6.8314]], grad_fn=<AddmmBackward>)
b: tensor([[3.8043]], grad_fn=<AddmmBackward>)
epoch [2/200], loss:556.0781
x:  tensor([[2.3186]], grad_fn=<AddmmBackward>)
y: tensor([[3.5541]], grad_fn=<AddmmBackward>)
b: tensor([[4.5475]], grad_fn=<AddmmBackward>)
epoch [3/200], loss:459.0005
x:  tensor([[0.6693]], grad_fn=<AddmmBackward>)
y: tensor([[0.3772]], grad_fn=<AddmmBackward>)
b: tensor([[4.8461]], grad_fn=<AddmmBackward>)
epoch [4/200], loss:370.9607
x:  tensor([[2.7533]], grad_fn=<AddmmBackward>)
y: tensor([[5.7296]], grad_fn=<AddmmBackward>)
b: tensor([[4.5765]], grad_fn=<AddmmBackward>)
epoch [5/200], loss:372.4210
x:  tensor([[4.4237]], grad_fn=<AddmmBackward>)
y: tensor([[8.5740]], grad_fn=<AddmmBackward>)
b: tensor([[4.0266]], grad_fn=<AddmmBackward>)
epoch [6/200], loss:266.3937
x:  tensor([[4.4563]], grad_fn=<AddmmBackward>)
y: tensor([[7.9394]], grad_fn=<AddmmBackward>)
b:

epoch [49/200], loss:6.0695
x:  tensor([[-1.5539]], grad_fn=<AddmmBackward>)
y: tensor([[-1.1238]], grad_fn=<AddmmBackward>)
b: tensor([[2.4031]], grad_fn=<AddmmBackward>)
epoch [50/200], loss:3.7880
x:  tensor([[0.9839]], grad_fn=<AddmmBackward>)
y: tensor([[1.0697]], grad_fn=<AddmmBackward>)
b: tensor([[2.4631]], grad_fn=<AddmmBackward>)
epoch [51/200], loss:3.5524
x:  tensor([[1.4049]], grad_fn=<AddmmBackward>)
y: tensor([[0.4193]], grad_fn=<AddmmBackward>)
b: tensor([[2.3596]], grad_fn=<AddmmBackward>)
epoch [52/200], loss:3.2062
x:  tensor([[-2.9828]], grad_fn=<AddmmBackward>)
y: tensor([[0.5620]], grad_fn=<AddmmBackward>)
b: tensor([[2.3142]], grad_fn=<AddmmBackward>)
epoch [53/200], loss:2.6747
x:  tensor([[0.2602]], grad_fn=<AddmmBackward>)
y: tensor([[1.1943]], grad_fn=<AddmmBackward>)
b: tensor([[2.2455]], grad_fn=<AddmmBackward>)
epoch [54/200], loss:2.4427
x:  tensor([[1.4228]], grad_fn=<AddmmBackward>)
y: tensor([[1.1136]], grad_fn=<AddmmBackward>)
b: tensor([[2.2487]], gr

epoch [97/200], loss:-0.6882
x:  tensor([[-1.8846]], grad_fn=<AddmmBackward>)
y: tensor([[-1.0278]], grad_fn=<AddmmBackward>)
b: tensor([[1.1406]], grad_fn=<AddmmBackward>)
epoch [98/200], loss:-0.7776
x:  tensor([[-1.4141]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6758]], grad_fn=<AddmmBackward>)
b: tensor([[1.1499]], grad_fn=<AddmmBackward>)
epoch [99/200], loss:-0.8156
x:  tensor([[-1.7550]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6765]], grad_fn=<AddmmBackward>)
b: tensor([[1.3254]], grad_fn=<AddmmBackward>)
epoch [100/200], loss:-0.8656
x:  tensor([[0.1563]], grad_fn=<AddmmBackward>)
y: tensor([[0.3091]], grad_fn=<AddmmBackward>)
b: tensor([[1.0348]], grad_fn=<AddmmBackward>)
epoch [101/200], loss:-0.8331
x:  tensor([[-1.5207]], grad_fn=<AddmmBackward>)
y: tensor([[-0.8433]], grad_fn=<AddmmBackward>)
b: tensor([[0.9983]], grad_fn=<AddmmBackward>)
epoch [102/200], loss:-0.8366
x:  tensor([[-1.6277]], grad_fn=<AddmmBackward>)
y: tensor([[-1.5326]], grad_fn=<AddmmBackward>)
b: tenso

epoch [145/200], loss:-0.9730
x:  tensor([[0.4581]], grad_fn=<AddmmBackward>)
y: tensor([[0.5037]], grad_fn=<AddmmBackward>)
b: tensor([[0.5916]], grad_fn=<AddmmBackward>)
epoch [146/200], loss:-0.9767
x:  tensor([[0.4679]], grad_fn=<AddmmBackward>)
y: tensor([[0.6130]], grad_fn=<AddmmBackward>)
b: tensor([[0.5188]], grad_fn=<AddmmBackward>)
epoch [147/200], loss:-0.9780
x:  tensor([[0.5528]], grad_fn=<AddmmBackward>)
y: tensor([[0.4188]], grad_fn=<AddmmBackward>)
b: tensor([[0.4858]], grad_fn=<AddmmBackward>)
epoch [148/200], loss:-0.9802
x:  tensor([[-1.5447]], grad_fn=<AddmmBackward>)
y: tensor([[-1.4834]], grad_fn=<AddmmBackward>)
b: tensor([[0.5475]], grad_fn=<AddmmBackward>)
epoch [149/200], loss:-0.9777
x:  tensor([[-1.5833]], grad_fn=<AddmmBackward>)
y: tensor([[-1.3757]], grad_fn=<AddmmBackward>)
b: tensor([[0.4752]], grad_fn=<AddmmBackward>)
epoch [150/200], loss:-0.9793
x:  tensor([[-1.7180]], grad_fn=<AddmmBackward>)
y: tensor([[-1.3534]], grad_fn=<AddmmBackward>)
b: tensor

epoch [193/200], loss:-0.9934
x:  tensor([[0.2813]], grad_fn=<AddmmBackward>)
y: tensor([[0.1817]], grad_fn=<AddmmBackward>)
b: tensor([[0.3252]], grad_fn=<AddmmBackward>)
epoch [194/200], loss:-0.9949
x:  tensor([[0.2475]], grad_fn=<AddmmBackward>)
y: tensor([[0.2223]], grad_fn=<AddmmBackward>)
b: tensor([[0.3906]], grad_fn=<AddmmBackward>)
epoch [195/200], loss:-0.9944
x:  tensor([[-2.3200]], grad_fn=<AddmmBackward>)
y: tensor([[-1.2978]], grad_fn=<AddmmBackward>)
b: tensor([[0.2987]], grad_fn=<AddmmBackward>)
epoch [196/200], loss:-0.9949
x:  tensor([[0.1250]], grad_fn=<AddmmBackward>)
y: tensor([[0.1131]], grad_fn=<AddmmBackward>)
b: tensor([[0.2291]], grad_fn=<AddmmBackward>)
epoch [197/200], loss:-0.9953
x:  tensor([[-2.2960]], grad_fn=<AddmmBackward>)
y: tensor([[-1.4072]], grad_fn=<AddmmBackward>)
b: tensor([[0.3195]], grad_fn=<AddmmBackward>)
epoch [198/200], loss:-0.9950
x:  tensor([[0.1432]], grad_fn=<AddmmBackward>)
y: tensor([[0.2266]], grad_fn=<AddmmBackward>)
b: tensor([

In [35]:
data3=[]

for i in range(5):
    img = Image.new('RGB', (28, 28), color = 'white')
    #img = np.reshape(img,(28,28,1))
    #print(np.shape(img))
    row,col,ch= np.shape(img)
    mean = random.randint(1,6)
    var = random.random()
    sigma = var**0.5
    gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
    noisy_pos1 = img + gauss_pos1
    im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
    draw = ImageDraw.Draw(im2_pos1)
    draw.rectangle([0,0,5,5],fill='red',outline='red')
    #im2_pos1.show()
    #print(np.shape(np.transpose(np.asarray(im2_pos1),(2,0,1))))
    im2_pos1 = np.reshape(im2_pos1,(28,28,3))

    data3.append(np.array(np.transpose(np.asarray(im2_pos1),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,28,28)))

x:  tensor([[-2.4450]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.2881]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.4508]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.4057]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.3064]], grad_fn=<AddmmBackward>)
